In [29]:
import csv
import numpy as np
import folium

def build_dist_mat(input_list):
    n = 11
    dist_mat = np.zeros([n, n])
    for i in range(n):
        for j in range(i + 1, n):
            d = input_list[i, :] - input_list[j, :]
            # 计算点积
            dist_mat[i, j] = np.dot(d, d)
            dist_mat[j, i] = dist_mat[i, j]
    return dist_mat

def tsp_dp(graph):
    num_cities = len(graph)
    
    # dp[mask][i]表示已经访问过的城市集合为mask，当前所在的城市为i时的最短路径长度
    dp = [[float('inf')] * num_cities for _ in range(1 << num_cities)]
    
    # path[mask][i]表示从起始城市到当前城市的最短路径上的城市顺序
    path = [[[] for _ in range(num_cities)] for _ in range(1 << num_cities)]
    
    # 初始化，从起始城市出发，到达自身的距离为0
    dp[1][0] = 0
    path[1][0] = [0]
    
    # 遍历所有的状态
    for mask in range(1, 1 << num_cities):
        for i in range(num_cities):
            # 如果当前城市不在mask中，则跳过
            if not (mask & (1 << i)):
                continue
            
            # 遍历所有可能的上一个城市
            for j in range(num_cities):
                # 如果上一个城市在mask中，且当前城市与上一个城市不同
                if mask & (1 << j) and i != j:
                    if dp[mask][i] > dp[mask ^ (1 << i)][j] + graph[j][i]:
                        dp[mask][i] = dp[mask ^ (1 << i)][j] + graph[j][i]
                        path[mask][i] = path[mask ^ (1 << i)][j] + [i]
    
    # 找到最短路径
    mask = (1 << num_cities) - 1
    min_length = min(dp[mask][i] + graph[i][0] for i in range(1, num_cities))
    min_path = path[mask][[i for i in range(1, num_cities) if dp[mask][i] + graph[i][0] == min_length][0]] + [0]
    
    return min_length, min_path


sites = []

# Replace 'your_file_path.csv' with the actual path to your CSV file
with open('sites.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        latitude = float(row[1])
        longitude = float(row[2])
        sites.append([longitude, latitude])

sites_array = np.array(sites)
# 示例：使用邻接矩阵表示城市之间的距离
graph = build_dist_mat(sites_array)

result = tsp_dp(graph)
print("shortest path length:", result[0])

path = result[1]
print("shortest path:", path)
result_pos_list = sites_array[path, :]

result_pos_list[:, [0, 1]] = result_pos_list[:, [1, 0]]

m = folium.Map(location=(49.284, -123.125), zoom_start=14, tiles="cartodb positron")
for i in range(1,len(result_pos_list)):
    folium.Marker(
        location=result_pos_list[i],
        tooltip= path[i],
        icon=folium.Icon(icon="cloud",color="orange"),
    ).add_to(m)
folium.PolyLine(result_pos_list, tooltip="Route").add_to(m)
m

shortest path length: 0.0007155068920005456
shortest path: [0, 10, 6, 1, 5, 2, 7, 3, 4, 9, 8, 0]
